In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_summary_tables = pd.read_excel("summary_tables_floods_all_aois.xlsx")
df_summary_tables

,activation_code,event_name,reason,flood_type,activator,eventTime,activationTime,gdacsId,countries,reportLink,aoiName,recent_event_extent_ha,max_event_ha,population_affected_recent,population_affected_max,source,notes_population_difference
0,EMSR775,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Coastal flood,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...,Viksdalen,0.2,0.2,NaN,NaN,copernicus,NaN
1,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Coastal flood,Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...,Valencia province,6702.0,17741.5,550.0,190000.0,copernicus,NaN
2,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Coastal flood,Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...,Plana de Utiel,102.8,102.8,1300.0,1300.0,copernicus,NaN
3,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Coastal flood,Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...,Horta Sud,646.4,2407.4,62000.0,62000.0,copernicus,NaN
4,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Coastal flood,Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...,Algemesi,568.8,568.8,11000.0,11000.0,copernicus,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Coastal flood,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN,Spazzate-Sassatelli,NaN,NaN,NaN,200.0,copernicus,NaN
191,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Coastal flood,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN,Lamone-Faenza,NaN,NaN,NaN,400.0,copernicus,NaN
192,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Coastal flood,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN,Bagnacavallo,NaN,NaN,NaN,150.0,copernicus,NaN
193,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Coastal flood,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN,Idice-Molinella,NaN,NaN,NaN,70.0,copernicus,NaN


In [10]:
id_vars = ["activation_code", "aoiName"]
value_vars = [
    "recent_event_extent_ha",
    "max_event_ha",
    "population_affected_recent",
    "population_affected_max",
]

melted_df = pd.melt(df_summary_tables, id_vars=id_vars, value_vars=value_vars, var_name="subcategory_unit", value_name="affected")
melted_df

,activation_code,aoiName,subcategory_unit,affected
0,EMSR775,Viksdalen,recent_event_extent_ha,0.2
1,EMSR773,Valencia province,recent_event_extent_ha,6702.0
2,EMSR773,Plana de Utiel,recent_event_extent_ha,102.8
3,EMSR773,Horta Sud,recent_event_extent_ha,646.4
4,EMSR773,Algemesi,recent_event_extent_ha,568.8
...,...,...,...,...
775,EMSR659,Spazzate-Sassatelli,population_affected_max,200.0
776,EMSR659,Lamone-Faenza,population_affected_max,400.0
777,EMSR659,Bagnacavallo,population_affected_max,150.0
778,EMSR659,Idice-Molinella,population_affected_max,70.0


In [11]:
melted_df.to_excel("summary_tables_for_merge.xlsx", index=False)

In [19]:
melted_df["unit_measurement"] = melted_df["subcategory_unit"].apply(
    lambda x: "hectares" if x in ["recent_event_extent_ha", "max_event_ha"] else "number"
)
melted_df

,activation_code,aoiName,subcategory_unit,affected,unit_measurement
0,EMSR775,Viksdalen,recent_event_extent_ha,0.2,hectares
1,EMSR773,Valencia province,recent_event_extent_ha,6702.0,hectares
2,EMSR773,Plana de Utiel,recent_event_extent_ha,102.8,hectares
3,EMSR773,Horta Sud,recent_event_extent_ha,646.4,hectares
4,EMSR773,Algemesi,recent_event_extent_ha,568.8,hectares
...,...,...,...,...,...
775,EMSR659,Spazzate-Sassatelli,population_affected_max,200.0,number
776,EMSR659,Lamone-Faenza,population_affected_max,400.0,number
777,EMSR659,Bagnacavallo,population_affected_max,150.0,number
778,EMSR659,Idice-Molinella,population_affected_max,70.0,number


In [20]:
melted_df["category_unit"] = melted_df["subcategory_unit"].apply(
    lambda x: "Flood extent" if x in ["recent_event_extent_ha", "max_event_ha"] else "Population"
)
melted_df

,activation_code,aoiName,subcategory_unit,affected,unit_measurement,category_unit
0,EMSR775,Viksdalen,recent_event_extent_ha,0.2,hectares,Flood extent
1,EMSR773,Valencia province,recent_event_extent_ha,6702.0,hectares,Flood extent
2,EMSR773,Plana de Utiel,recent_event_extent_ha,102.8,hectares,Flood extent
3,EMSR773,Horta Sud,recent_event_extent_ha,646.4,hectares,Flood extent
4,EMSR773,Algemesi,recent_event_extent_ha,568.8,hectares,Flood extent
...,...,...,...,...,...,...
775,EMSR659,Spazzate-Sassatelli,population_affected_max,200.0,number,Population
776,EMSR659,Lamone-Faenza,population_affected_max,400.0,number,Population
777,EMSR659,Bagnacavallo,population_affected_max,150.0,number,Population
778,EMSR659,Idice-Molinella,population_affected_max,70.0,number,Population


In [21]:
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   activation_code   780 non-null    object
 1   aoiName           768 non-null    object
 2   subcategory_unit  780 non-null    object
 3   affected          612 non-null    object
 4   unit_measurement  780 non-null    object
 5   category_unit     780 non-null    object
dtypes: object(6)
memory usage: 36.7+ KB


In [22]:
new_column_order = [
    "activation_code",
    "aoiName",
    "category_unit", 
    "unit_measurement",
    "subcategory_unit",
    "affected"
]

melted_df = melted_df[new_column_order]
melted_df

,activation_code,aoiName,category_unit,unit_measurement,subcategory_unit,affected
0,EMSR775,Viksdalen,Flood extent,hectares,recent_event_extent_ha,0.2
1,EMSR773,Valencia province,Flood extent,hectares,recent_event_extent_ha,6702.0
2,EMSR773,Plana de Utiel,Flood extent,hectares,recent_event_extent_ha,102.8
3,EMSR773,Horta Sud,Flood extent,hectares,recent_event_extent_ha,646.4
4,EMSR773,Algemesi,Flood extent,hectares,recent_event_extent_ha,568.8
...,...,...,...,...,...,...
775,EMSR659,Spazzate-Sassatelli,Population,number,population_affected_max,200.0
776,EMSR659,Lamone-Faenza,Population,number,population_affected_max,400.0
777,EMSR659,Bagnacavallo,Population,number,population_affected_max,150.0
778,EMSR659,Idice-Molinella,Population,number,population_affected_max,70.0


In [12]:
df_detailed = pd.read_excel("all_aois_clean_normalized.xlsx")
df_detailed

,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,indication of rural area
0,EMSR775,Viksdalen,11,DEL,0,2024-11-05T19:18:42.042642,Land use,hectares,Land use_Heterogeneous agricultural areas _(ha),heterogenous agricultural areas,agriculture,0.2,NaN
1,EMSR775,Viksdalen,11,DEL,0,2024-11-05T19:18:42.042642,Facilities,kilometres,Facilities_Local pipelines and cables_(km),local pipelines and cables,pipelines and communication infrastructure,0.1,NaN
2,EMSR773,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Built-up,hectares,"Built-up_School, university and research build...","schools, universities, and research buildings",public service and educational buildings,51.1,NaN
3,EMSR773,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Other_(ha),unclassified land,other land,4573.0,NaN
4,EMSR773,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Forests _(ha),forests,forests,47.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,EMSR657,DaugavaRiver,1,DEL,2,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Local Road_(km),local roads,roads,6.8,NaN
2588,EMSR657,DaugavaRiver,1,DEL,2,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Primary Road_(km),primary roads,roads,0.3,NaN
2589,EMSR657,DaugavaRiver,1,DEL,2,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Secondary Road_(km),secondary roads,roads,19.1,NaN
2590,EMSR657,DaugavaRiver,1,DEL,2,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Airfield runways_(km),airfield runways,aviation infrastructure,0.9,NaN


In [23]:
concatenated_df = pd.concat([df_detailed, melted_df], ignore_index=True)
concatenated_df

,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,indication of rural area
0,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Land use,hectares,Land use_Heterogeneous agricultural areas _(ha),heterogenous agricultural areas,agriculture,0.2,NaN
1,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Facilities,kilometres,Facilities_Local pipelines and cables_(km),local pipelines and cables,pipelines and communication infrastructure,0.1,NaN
2,EMSR773,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Built-up,hectares,"Built-up_School, university and research build...","schools, universities, and research buildings",public service and educational buildings,51.1,NaN
3,EMSR773,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Other_(ha),unclassified land,other land,4573.0,NaN
4,EMSR773,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Forests _(ha),forests,forests,47.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3367,EMSR659,Spazzate-Sassatelli,NaN,NaN,NaN,NaN,Population,number,population_affected_max,NaN,NaN,200.0,NaN
3368,EMSR659,Lamone-Faenza,NaN,NaN,NaN,NaN,Population,number,population_affected_max,NaN,NaN,400.0,NaN
3369,EMSR659,Bagnacavallo,NaN,NaN,NaN,NaN,Population,number,population_affected_max,NaN,NaN,150.0,NaN
3370,EMSR659,Idice-Molinella,NaN,NaN,NaN,NaN,Population,number,population_affected_max,NaN,NaN,70.0,NaN


In [27]:
concatenated_df_sorted = concatenated_df.sort_values(by=["activation_code", "aoiName"], ascending=False)
concatenated_df_sorted

,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,indication of rural area
0,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Land use,hectares,Land use_Heterogeneous agricultural areas _(ha),heterogenous agricultural areas,agriculture,0.2,NaN
1,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Facilities,kilometres,Facilities_Local pipelines and cables_(km),local pipelines and cables,pipelines and communication infrastructure,0.1,NaN
2592,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,NaN,0.2,NaN
2787,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,NaN,0.2,NaN
2982,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2591,EMSR657,DaugavaRiver,1.0,DEL,2.0,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Long-distance railways_(km),long-distance railways,railway infrastructure,1.0,NaN
2786,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,NaN,NaN,NaN
2981,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,NaN,NaN,NaN
3176,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,NaN,NaN,NaN


In [29]:
concatenated_df_sorted.to_excel("merged_data_copernicus.xlsx", index=False)

In [30]:
df_for_merge = pd.read_excel("to_be_merged.xlsx")

In [31]:
df_for_merge

,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,Unnamed: 12,Unnamed: 13
0,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Land use,hectares,Land use_Heterogeneous agricultural areas _(ha),heterogenous agricultural areas,agriculture,0.2,NaN,NaN
1,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Facilities,kilometres,Facilities_Local pipelines and cables_(km),local pipelines and cables,pipelines and communication infrastructure,0.1,NaN,NaN
2,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,NaN,0.2,NaN,NaN
3,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,NaN,0.2,NaN,NaN
4,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3361,EMSR657,DaugavaRiver,1.0,DEL,2.0,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Long-distance railways_(km),long-distance railways,railway infrastructure,1.0,NaN,NaN
3362,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,NaN,NaN,NaN,NaN
3363,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,NaN,7307.3,check again based on answers on flood trace an...,NaN
3364,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,NaN,NaN,NaN,NaN


In [32]:
df_summary_tables

,activation_code,event_name,reason,flood_type,activator,eventTime,activationTime,gdacsId,countries,reportLink,aoiName,recent_event_extent_ha,max_event_ha,population_affected_recent,population_affected_max,source,notes_population_difference
0,EMSR775,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Coastal flood,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...,Viksdalen,0.2,0.2,NaN,NaN,copernicus,NaN
1,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Coastal flood,Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...,Valencia province,6702.0,17741.5,550.0,190000.0,copernicus,NaN
2,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Coastal flood,Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...,Plana de Utiel,102.8,102.8,1300.0,1300.0,copernicus,NaN
3,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Coastal flood,Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...,Horta Sud,646.4,2407.4,62000.0,62000.0,copernicus,NaN
4,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Coastal flood,Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...,Algemesi,568.8,568.8,11000.0,11000.0,copernicus,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Coastal flood,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN,Spazzate-Sassatelli,NaN,NaN,NaN,200.0,copernicus,NaN
191,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Coastal flood,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN,Lamone-Faenza,NaN,NaN,NaN,400.0,copernicus,NaN
192,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Coastal flood,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN,Bagnacavallo,NaN,NaN,NaN,150.0,copernicus,NaN
193,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Coastal flood,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN,Idice-Molinella,NaN,NaN,NaN,70.0,copernicus,NaN


In [34]:
df_summary_tables.drop(columns=["aoiName", "recent_event_extent_ha", "max_event_ha", "population_affected_recent", "population_affected_max", "source", "notes_population_difference"], inplace=True)

In [37]:
df_summary_tables.drop(columns=["flood_type"], inplace=True)
df_summary_tables

,activation_code,event_name,reason,activator,eventTime,activationTime,gdacsId,countries,reportLink
0,EMSR775,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
1,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...
2,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...
3,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...
4,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...
...,...,...,...,...,...,...,...,...,...
190,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN
191,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN
192,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN
193,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,FL1101976,Italy,NaN


In [39]:
df_summary_tables_unique = df_summary_tables.drop_duplicates(subset="activation_code", keep="first")

In [41]:
df_summary_tables_unique

,activation_code,event_name,reason,activator,eventTime,activationTime,gdacsId,countries,reportLink
0,EMSR775,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
1,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,FL1102983,Spain,https://storymaps.arcgis.com/stories/9ff57e932...
33,EMSR771,"Flood in Emilia- Romagna, Italy",Italy has been hit in recent days by a severe ...,Italy|Presidency of the Council of Ministers -...,2024-10-20T12:00:00,2024-10-20T08:24:00,FL1102961,Italy,https://storymaps.arcgis.com/stories/b95e2210e...
39,EMSR770,Flood in Bosnia and Herzegovina,"On the 4 October 2024, a flood is reported to ...",Bosnia and Herzegovina|Protection and Rescue S...,2024-10-04T00:00:00,2024-10-04T15:59:00,NaN,Bosnia and Herzegovina,https://storymaps.arcgis.com/stories/8a4d435dd...
42,EMSR768,Flood in Croatia and Slovenia,"On the 4 October 2024 in the afternoon, the pe...",Croatia|National Protection and Rescue Directo...,2024-10-04T14:00:00,2024-10-04T09:35:00,NaN,Croatia,https://storymaps.arcgis.com/stories/47a00f810...
44,EMSR766,Flood in Croatia,"On the 25 September 2024 at 12:00, top of the ...",Croatia|National Protection and Rescue Directo...,2024-09-25T12:00:00,2024-09-24T11:53:00,NaN,Croatia,https://storymaps.arcgis.com/stories/4b09f8c48...
46,EMSR764,"Flood in Lower Austria, Austria","On the 14 September 2024, heavy reain with hug...","EC Services|DG-ECHO, DG DEFIS, DG DEVCO, DG HO...",2024-09-14T12:00:00,2024-09-22T08:19:00,NaN,Austria,https://storymaps.arcgis.com/stories/dba0e5785...
50,EMSR763,Flood in Germany,On the 22 September a flood event is forecast ...,Germany|Bundesamt fur Bevolkerungsschutz und K...,2024-09-22T12:00:00,2024-09-20T13:01:00,NaN,Germany,https://storymaps.arcgis.com/stories/fd7279bc4...
51,EMSR762,"Flood in Emilia-Romagna, Italy","Since the early morning of 18 September 2024, ...",Italy|Presidency of the Council of Ministers -...,2024-09-18T06:00:00,2024-09-18T14:58:00,FL1102833,Italy,https://storymaps.arcgis.com/stories/c5d511391...
57,EMSR761,"Flood in Dresden, Germany",On September 18 of September 2024 at 13:00 UTC...,Autorised User,2024-09-18T12:00:00,2024-09-18T13:00:00,NaN,Germany,https://storymaps.arcgis.com/stories/071ffcf4f...


In [42]:
df_for_merge

,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,Unnamed: 12,Unnamed: 13
0,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Land use,hectares,Land use_Heterogeneous agricultural areas _(ha),heterogenous agricultural areas,agriculture,0.2,NaN,NaN
1,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Facilities,kilometres,Facilities_Local pipelines and cables_(km),local pipelines and cables,pipelines and communication infrastructure,0.1,NaN,NaN
2,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,NaN,0.2,NaN,NaN
3,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,NaN,0.2,NaN,NaN
4,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3361,EMSR657,DaugavaRiver,1.0,DEL,2.0,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Long-distance railways_(km),long-distance railways,railway infrastructure,1.0,NaN,NaN
3362,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,NaN,NaN,NaN,NaN
3363,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,NaN,7307.3,check again based on answers on flood trace an...,NaN
3364,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,NaN,NaN,NaN,NaN


In [44]:
merged_df = pd.merge(df_for_merge, df_summary_tables, on="activation_code", how="left")
merged_df

,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,...,Unnamed: 12,Unnamed: 13,event_name,reason,activator,eventTime,activationTime,gdacsId,countries,reportLink
0,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Land use,hectares,Land use_Heterogeneous agricultural areas _(ha),heterogenous agricultural areas,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
1,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Facilities,kilometres,Facilities_Local pipelines and cables_(km),local pipelines and cables,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
2,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
3,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
4,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42670,EMSR657,DaugavaRiver,1.0,DEL,2.0,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Long-distance railways_(km),long-distance railways,...,NaN,NaN,"Flood in Daugava River valley, Latvia",As of 30 March 2023 a flood due to melted snow...,Latvia|Latvian State Fire and Rescue Service,2023-03-30T00:00:00,2023-03-31T11:49:00,FL1101921,Latvia,NaN
42671,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,...,NaN,NaN,"Flood in Daugava River valley, Latvia",As of 30 March 2023 a flood due to melted snow...,Latvia|Latvian State Fire and Rescue Service,2023-03-30T00:00:00,2023-03-31T11:49:00,FL1101921,Latvia,NaN
42672,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,...,check again based on answers on flood trace an...,NaN,"Flood in Daugava River valley, Latvia",As of 30 March 2023 a flood due to melted snow...,Latvia|Latvian State Fire and Rescue Service,2023-03-30T00:00:00,2023-03-31T11:49:00,FL1101921,Latvia,NaN
42673,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,...,NaN,NaN,"Flood in Daugava River valley, Latvia",As of 30 March 2023 a flood due to melted snow...,Latvia|Latvian State Fire and Rescue Service,2023-03-30T00:00:00,2023-03-31T11:49:00,FL1101921,Latvia,NaN


In [47]:
df_no_duplicates = merged_df.drop_duplicates(subset=["activation_code", "aoiName", "unit_measurement", "subcategory_unit"])

In [48]:
df_no_duplicates

,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,...,Unnamed: 12,Unnamed: 13,event_name,reason,activator,eventTime,activationTime,gdacsId,countries,reportLink
0,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Land use,hectares,Land use_Heterogeneous agricultural areas _(ha),heterogenous agricultural areas,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
1,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Facilities,kilometres,Facilities_Local pipelines and cables_(km),local pipelines and cables,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
2,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
3,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
4,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,...,NaN,NaN,Flood in Norway,On the 31 October 2024 a plain flood is foreca...,Norway|Directorate for Civil Protection and Em...,2024-10-31T23:00:00,2024-10-30T19:04:00,NaN,Norway,https://storymaps.arcgis.com/stories/8d09aca12...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42670,EMSR657,DaugavaRiver,1.0,DEL,2.0,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Long-distance railways_(km),long-distance railways,...,NaN,NaN,"Flood in Daugava River valley, Latvia",As of 30 March 2023 a flood due to melted snow...,Latvia|Latvian State Fire and Rescue Service,2023-03-30T00:00:00,2023-03-31T11:49:00,FL1101921,Latvia,NaN
42671,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,...,NaN,NaN,"Flood in Daugava River valley, Latvia",As of 30 March 2023 a flood due to melted snow...,Latvia|Latvian State Fire and Rescue Service,2023-03-30T00:00:00,2023-03-31T11:49:00,FL1101921,Latvia,NaN
42672,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,...,check again based on answers on flood trace an...,NaN,"Flood in Daugava River valley, Latvia",As of 30 March 2023 a flood due to melted snow...,Latvia|Latvian State Fire and Rescue Service,2023-03-30T00:00:00,2023-03-31T11:49:00,FL1101921,Latvia,NaN
42673,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,...,NaN,NaN,"Flood in Daugava River valley, Latvia",As of 30 March 2023 a flood due to melted snow...,Latvia|Latvian State Fire and Rescue Service,2023-03-30T00:00:00,2023-03-31T11:49:00,FL1101921,Latvia,NaN


In [50]:
df_no_duplicates_for_checking = df_no_duplicates.drop(columns=["event_name", "reason", "activator", "eventTime", "activationTime", "gdacsId", "countries", "reportLink"])

In [53]:
df_no_duplicates_for_checking.reset_index(drop=True)

,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,Unnamed: 12,Unnamed: 13
0,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Land use,hectares,Land use_Heterogeneous agricultural areas _(ha),heterogenous agricultural areas,agriculture,0.2,NaN,NaN
1,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Facilities,kilometres,Facilities_Local pipelines and cables_(km),local pipelines and cables,pipelines and communication infrastructure,0.1,NaN,NaN
2,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,NaN,0.2,NaN,NaN
3,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,NaN,0.2,NaN,NaN
4,EMSR775,Viksdalen,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3361,EMSR657,DaugavaRiver,1.0,DEL,2.0,2023-04-03T16:23:00,Transportation,kilometres,Transportation_Long-distance railways_(km),long-distance railways,railway infrastructure,1.0,NaN,NaN
3362,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,recent_event_extent_ha,NaN,NaN,NaN,NaN,NaN
3363,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Flood extent,hectares,max_event_ha,NaN,NaN,7307.3,check again based on answers on flood trace an...,NaN
3364,EMSR657,DaugavaRiver,NaN,NaN,NaN,NaN,Population,number,population_affected_recent,NaN,NaN,NaN,NaN,NaN


In [56]:
different = df_no_duplicates_for_checking[~df_no_duplicates_for_checking['subcategory_unit'].isin(df_for_merge['subcategory_unit'])]
print("Rows with different activation_code:\n", different)

Rows with different activation_code:
 Empty DataFrame
Columns: [activation_code, aoiName, aoiNumber, type, monitoring, deliveryTime, category_unit, unit_measurement, subcategory_unit, normalized_category_v1, normalized_category_v2, affected, Unnamed: 12, Unnamed: 13]
Index: []


In [57]:
df_no_duplicates.to_excel("all_aois_details_summaries_activations.xlsx", index=False)